[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ant-research/EasyTemporalPointProcess/blob/main/notebooks/easytpp_3_train_eval.ipynb)

# Tutorial 3：Training and Evaluation Pipeline in EasyTPP

In this tutorial, we'll walk through the complete process of training and evaluating temporal point process (TPP) models using the **EasyTPP** framework.

In this notebook, we will cover the following key aspects:
- **Data Preparation**: Loading and preprocessing event sequence data.
- **Model Training**: Configuring and training a Neural Hawkes Process (NHP) model.
- **Model Evaluation**: Assessing model performance using various metrics.
- **Visualization**: Analyzing and visualizing model predictions and results.

We begin by installing the package

In [18]:
! pip install --upgrade easy-tpp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: easy-tpp
    Found existing installation: easy-tpp 0.1.0
    Uninstalling easy-tpp-0.1.0:
      Successfully uninstalled easy-tpp-0.1.0


# Step 1: setup the config file

The EasyTPP framework uses a YAML-based configuration system that consists of two main components:
- **data configuration**, which specifies the data sources and their formats. It defines where to find the training, validation, and test datasets (see below as an example). The full explanation of the dataset can be found in the previous tutorial [EasyTPP-Dataset](https://github.com/ant-research/EasyTemporalPointProcess/blob/main/notebooks/easytpp_1_dataset.ipynb).
- **model configuration**, which defines the model architecture, hyperparameters, and training settings.

Let's first look at the data configuration:

In [1]:
data_config = """
data:
  taxi:
    data_format: json
    train_dir:  ./data/taxi/train.json
    valid_dir:  ./data/taxi/dev.json
    test_dir:  ./data/taxi/test.json

"""

The model configuration specifies the neural network architecture and training hyperparameters. It is structured into two main sections: base configuration for general settings and trainer-specific configuration for detailed training parameters:


In [2]:
model_config = """
NHP_train:
  base_config:
    stage: train
    backend: torch
    dataset_id: taxi
    runner_id: std_tpp
    model_id: NHP # model name
    base_dir: './checkpoints/'
  trainer_config:
    batch_size: 256
    max_epoch: 2
    shuffle: True
    optimizer: adam
    learning_rate: 1.e-3
    valid_freq: 1
    use_tfb: False
    metrics: [ 'acc', 'rmse' ]
    seed: 2019
    gpu: -1
"""

As an illustrative example, we write the YAML content to a file yaml_content as below. Note that `runner_config` is the pipeline configuration ID that tells EasyTPP which configuration to use. We combine the data configuration
and model configuration into a single YAML file. This file will be used to initialize the model runner, which will handle the training process based on the specified parameters.

In [3]:
yaml_content = """
pipeline_config_id: runner_config

data:
    taxi:
        data_format: json
        train_dir: easytpp/taxi  # ./data/taxi/train.json
        valid_dir: easytpp/taxi  # ./data/taxi/dev.json
        test_dir: easytpp/taxi   # ./data/taxi/test.json
        data_specs:
          num_event_types: 10
          pad_token_id: 10
          padding_side: right


NHP_train:
  base_config:
    stage: train
    backend: torch
    dataset_id: taxi
    runner_id: std_tpp
    model_id: NHP # model name
    base_dir: './checkpoints/'
  trainer_config:
    batch_size: 256
    max_epoch: 2
    shuffle: False
    optimizer: adam
    learning_rate: 1.e-3
    valid_freq: 1
    use_tfb: True
    metrics: [ 'acc', 'rmse' ]
    seed: 2019
    gpu: -1
  model_config:
    hidden_size: 32
    loss_integral_num_sample_per_step: 20
    thinning:
      num_seq: 10
      num_sample: 1
      num_exp: 500 # number of i.i.d. Exp(intensity_bound) draws at one time in thinning algorithm
      look_ahead_time: 10
      patience_counter: 5 # the maximum iteration used in adaptive thinning
      over_sample_rate: 5
      num_samples_boundary: 5
      dtime_max: 5
      num_step_gen: 1
"""

# Save the content to a file named config.yaml
with open("config.yaml", "w") as file:
    file.write(yaml_content)

## Step 2: Train the model

Then we can initialize the model runner using the configuration file we just created. The model runner will handle the training process based on the specified parameters in our configuration.

We'll use the `Config` class to build the configuration from the YAML file and specify an experiment ID (key of the `model_config`). Then, we'll create a `Runner` instance from this configuration and run the training process.

In [4]:
from easy_tpp.config_factory import Config
from easy_tpp.runner import Runner

config = Config.build_from_yaml_file('./config.yaml', experiment_id='NHP_train')

model_runner = Runner.build_from_config(config)

model_runner.run()

2025-04-06 05:42:18,953 - config.py[pid:60211;line:34:build_from_yaml_file] - CRITICAL: Load pipeline config class RunnerConfig
2025-04-06 05:42:18,957 - runner_config.py[pid:60211;line:151:update_config] - CRITICAL: train model NHP using CPU with torch backend
2025-04-06 05:42:18,970 - runner_config.py[pid:60211;line:35:__init__] - INFO: Save the config to ./checkpoints/60211_133800770625536_250406-054218/NHP_train_output.yaml
2025-04-06 05:42:18,972 - base_runner.py[pid:60211;line:176:save_log] - INFO: Save the log to ./checkpoints/60211_133800770625536_250406-054218/log


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0.2244252199397379 0.29228809611195583
min_dt: 0.000277777777777
max_dt: 5.721388888888889


Instructions for updating:
non-resource variables are not supported in the long term


2025-04-06 05:42:34,617 - tpp_runner.py[pid:60211;line:60:_init_model] - INFO: Num of model parameters 15252
2025-04-06 05:42:36,901 - base_runner.py[pid:60211;line:98:train] - INFO: Data 'taxi' loaded...
2025-04-06 05:42:36,902 - base_runner.py[pid:60211;line:103:train] - INFO: Start NHP training...
2025-04-06 05:42:39,574 - tpp_runner.py[pid:60211;line:96:_train_model] - INFO: [ Epoch 0 (train) ]: train loglike is -1.755373397054743, num_events is 50454
2025-04-06 05:42:45,689 - tpp_runner.py[pid:60211;line:107:_train_model] - INFO: [ Epoch 0 (valid) ]:  valid loglike is -1.6691416010202664, num_events is 7204, acc is 0.4415602443087174, rmse is 0.33315836060539783
2025-04-06 05:42:54,655 - tpp_runner.py[pid:60211;line:122:_train_model] - INFO: [ Epoch 0 (test) ]: test loglike is -1.6577474861303745, num_events is 14420, acc is 0.4467406380027739, rmse is 0.34015134195006963
2025-04-06 05:42:54,657 - tpp_runner.py[pid:60211;line:124:_train_model] - CRITICAL: current best loglike on v

# Step 2: Evaluate a model

After training completes, we can evaluate the model's performance on the test dataset. EasyTPP offers streamlined methods for loading trained models and conducting evaluations.

First, we need to locate the saved model directory within the `checkpoints` folder specified in our configuration. For example, the model might be stored at `./checkpoints/60211_133800770625536_250406-054218/models/saved_model`.

Next, we'll create a new configuration file specifically for evaluation. This evaluation process performs one-step prediction on the validation set defined in the `valid_dir` parameter.

For demonstration purposes, we'll use the same dataset for our evaluation as we used during training.

We write the YAML config for evaluation as below. This configuration specifies:
- The data sources for training, validation, and testing (only validation will be used, the other two are optional).
- The evaluation stage and backend framework
- Model parameters and batch size
- The path to our pretrained model from the previous training step

Note: Make sure the `thinning_config` is included in the configuration. Also, ensure that 'rmse' and 'acc' are specified in the `metrics` list under `trainer_config` to properly compute and return these evaluation metrics. The error in the previous run was due to missing 'rmse' in the metrics configuration.

In [6]:
eval_yaml ="""
pipeline_config_id: runner_config

data:
  taxi:
    data_format: json
    train_dir: easytpp/taxi  # ./data/taxi/train.json
    valid_dir: easytpp/taxi  # ./data/taxi/dev.json
    test_dir: easytpp/taxi   # ./data/taxi/test.json
    data_specs:
      num_event_types: 10
      pad_token_id: 10
      padding_side: right

NHP_eval:
  base_config:
    stage: eval
    backend: torch
    dataset_id: taxi
    runner_id: std_tpp
    base_dir: './checkpoints/'
    model_id: NHP
  trainer_config:
    batch_size: 256
    max_epoch: 1
    metrics: [ 'acc', 'rmse' ]
  model_config:
    hidden_size: 32
    use_ln: False
    seed: 2019
    gpu: -1
    pretrained_model_dir: ./checkpoints/60211_133800770625536_250406-054218/models/saved_model
    thinning:
      num_seq: 10
      num_sample: 1
      num_exp: 500 # number of i.i.d. Exp(intensity_bound) draws at one time in thinning algorithm
      look_ahead_time: 10
      patience_counter: 5 # the maximum iteration used in adaptive thinning
      over_sample_rate: 5
      num_samples_boundary: 5
      dtime_max: 5
"""

# Save the content to a file named config.yaml
with open("eval_config.yaml", "w") as file:
    file.write(eval_yaml)

We use the following script to evaluate the trained model. This will load the pretrained model from the specified directory and evaluate it on the test dataset


In [7]:
from easy_tpp.config_factory import Config
from easy_tpp.runner import Runner

config = Config.build_from_yaml_file('./eval_config.yaml', experiment_id='NHP_eval')

model_runner = Runner.build_from_config(config)

model_runner.run()

2025-04-06 05:43:28,529 - config.py[pid:60211;line:34:build_from_yaml_file] - CRITICAL: Load pipeline config class RunnerConfig
2025-04-06 05:43:28,532 - runner_config.py[pid:60211;line:151:update_config] - CRITICAL: validate model NHP using CPU with torch backend
2025-04-06 05:43:28,543 - runner_config.py[pid:60211;line:35:__init__] - INFO: Save the config to ./checkpoints/60211_133800770625536_250406-054328/NHP_eval_output.yaml
2025-04-06 05:43:28,545 - base_runner.py[pid:60211;line:176:save_log] - INFO: Save the log to ./checkpoints/60211_133800770625536_250406-054328/log
0.2244252199397379 0.29228809611195583
min_dt: 0.000277777777777
max_dt: 5.721388888888889
2025-04-06 05:43:29,769 - tpp_runner.py[pid:60211;line:60:_init_model] - INFO: Num of model parameters 15252
2025-04-06 05:43:29,773 - tpp_runner.py[pid:60211;line:81:_load_model] - CRITICAL: Load model from ./checkpoints/60211_133800770625536_250406-054218/models/saved_model
2025-04-06 05:43:30,305 - base_runner.py[pid:60211

np.float64(0.3325426308842327)

# Step 3: Generate predictions

After training and evaluation, we can generate predictions for new events given a sequence as `context`.

The generation process uses the trained model to perform the one-step (by default) or multi-step prediction.

In order to evaluate the prediction accuracy, we automatically mask the last-n events (where n depends on the prediction step count) and use them as ground truth (golden events) to compare against our model's predictions.

We'll use the same configuration file we used for evaluation, but we'll specify a different experiment ID for the prediction task.



In [8]:
gen_yaml ="""
pipeline_config_id: runner_config

data:
  taxi:
    data_format: json
    train_dir: easytpp/taxi  # ./data/taxi/train.json
    valid_dir: easytpp/taxi  # ./data/taxi/dev.json
    test_dir: easytpp/taxi   # ./data/taxi/test.json
    data_specs:
      num_event_types: 10
      pad_token_id: 10
      padding_side: right

NHP_gen:
  base_config:
    stage: gen
    backend: torch
    dataset_id: taxi
    runner_id: std_tpp
    base_dir: './checkpoints/'
    model_id: NHP
  trainer_config:
    batch_size: 256
    max_epoch: 1
    metrics: [ 'acc', 'rmse' ]
  model_config:
    hidden_size: 32
    use_ln: False
    seed: 2019
    gpu: -1
    pretrained_model_dir: ./checkpoints/60211_133800770625536_250406-054218/models/saved_model
    thinning:
      num_seq: 10
      num_sample: 1
      num_exp: 500 # number of i.i.d. Exp(intensity_bound) draws at one time in thinning algorithm
      look_ahead_time: 10
      patience_counter: 5 # the maximum iteration used in adaptive thinning
      over_sample_rate: 5
      num_samples_boundary: 5
      dtime_max: 5
      num_step_gen: 1
"""

# Save the content to a file named config.yaml
with open("gen_config.yaml", "w") as file:
    file.write(gen_yaml)

We continue to use a similar configuration file to launch the generation process. Note that it is necessary to explicitly specify `num_gen_step` in the thinning config. The results will be saved in a pickle file.

We acknowledge that the generation pipeline is still a work in progress. For instance, the multi-step sampling process can be further improved (ongoing work), and the output format could be enhanced to facilitate easier evaluation.

In [9]:
from easy_tpp.config_factory import Config
from easy_tpp.runner import Runner

config = Config.build_from_yaml_file('./gen_config.yaml', experiment_id='NHP_gen')

model_runner = Runner.build_from_config(config)

model_runner.run()

2025-04-06 05:43:48,495 - config.py[pid:60211;line:34:build_from_yaml_file] - CRITICAL: Load pipeline config class RunnerConfig
2025-04-06 05:43:48,498 - runner_config.py[pid:60211;line:151:update_config] - CRITICAL: predict model NHP using CPU with torch backend
2025-04-06 05:43:48,508 - runner_config.py[pid:60211;line:35:__init__] - INFO: Save the config to ./checkpoints/60211_133800770625536_250406-054348/NHP_gen_output.yaml
2025-04-06 05:43:48,510 - base_runner.py[pid:60211;line:176:save_log] - INFO: Save the log to ./checkpoints/60211_133800770625536_250406-054348/log
0.2244252199397379 0.29228809611195583
min_dt: 0.000277777777777
max_dt: 5.721388888888889
2025-04-06 05:43:49,245 - tpp_runner.py[pid:60211;line:60:_init_model] - INFO: Num of model parameters 15252
2025-04-06 05:43:49,249 - tpp_runner.py[pid:60211;line:81:_load_model] - CRITICAL: Load model from ./checkpoints/60211_133800770625536_250406-054218/models/saved_model
2025-04-06 05:43:49,777 - base_runner.py[pid:60211;l